In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import os

src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [2]:
routine_full = pd.read_csv('../../data/02_intermediate/routine_inter.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
routine_full.head(2)

,Sample,Prod,DateSampled,Grade,DateTested,HSID,Supplier,Gallons,FacilityName,SiteAddress,Test,Units,Method,Result,MinResult,MaxResult,Compliance
0,61916134,Gasoline,11/23/2015 12:00:00 AM,Mid Grade Unleaded,NaN,15752.0,Marathon Petroleum Lp,2498.6,Circle K #2723609,"198 Haywood Ln \r\nnashville, Tn 37211",Antiknock Index,,R+M/2,89.8,87,,Y
1,61916134,Gasoline,11/23/2015 12:00:00 AM,Mid Grade Unleaded,NaN,15752.0,Marathon Petroleum Lp,2498.6,Circle K #2723609,"198 Haywood Ln \r\nnashville, Tn 37211",API Gravity,,D4052,60.9,,,


In [4]:
routine_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1129219 entries, 0 to 1129218
Data columns (total 17 columns):
Sample          1129219 non-null object
Prod            1129217 non-null object
DateSampled     1129219 non-null object
Grade           1129217 non-null object
DateTested      27379 non-null object
HSID            391122 non-null float64
Supplier        949928 non-null object
Gallons         1122723 non-null object
FacilityName    1129180 non-null object
SiteAddress     1121979 non-null object
Test            1124054 non-null object
Units           1123818 non-null object
Method          1123818 non-null object
Result          1068726 non-null object
MinResult       994273 non-null object
MaxResult       1008953 non-null object
Compliance      1126891 non-null object
dtypes: float64(1), object(16)
memory usage: 146.5+ MB


#### Sample Variable

This is a unique identifier for each sample taken. As such, it should be a string (as it is according to info). 

#### Prod

There are two missing product entries. Since this is 2 entries in over 1million, I will drop the two missing entries

In [5]:
routine_full.dropna(subset=['Prod'], inplace=True)

#### DateSampled

This is entered as a float in the original dataset, but should be datetime

In [6]:
routine_full['DateSampled'] = pd.to_datetime(routine_full['DateSampled'])

KeyboardInterrupt: 

The expert we intereviewed mentioned that the time portion of the datetime column is innacurate. Because it doesn't add information I will remove it from the dataframe. 

In [ ]:
routine_full['DateSampled'] = pd.DatetimeIndex(routine_full['DateSampled']).normalize()

#### Grade

This variable does not need to be updated right now. 

#### DateTested

The expert we interviewed stated that teh datetested column is a relic from the old system. Most of the Datetested column is empty. Because of this we will drop it from our analysis

In [ ]:
routine_full.drop(['DateTested'], axis=1, inplace=True)

#### HSID

Our expert suggested dropping this from the sample. He believes it is a relic from the old system

In [ ]:
routine_full.drop(['HSID'], axis=1, inplace=True)

#### <font color='red'>Supplier</font>

~15% of the supplier column for the entire dataset is empty. 

In [ ]:
routine_full['Supplier'].isna().sum()/len(routine_full)

Since we are interested in Gasoline and Diesel let's check out the supplier na count for these two products seperatly

~15% of gasoline supplier column is N/A

In [ ]:
gas = routine_full.loc[routine_full['Prod'] == 'Gasoline']
gas['Supplier'].isna().sum()/len(gas)

In [ ]:
gas[gas['Supplier'].isna()]

~ around 6% of diesel supplier column is N/A

In [ ]:
diesel = routine_full.loc[routine_full['Prod'] == 'Diesel']
diesel['Supplier'].isna().sum()/len(gas)

#### Gallons

Known Problems
1. Contains a number of Nan Values
1. Many values contain inches (relic from old system). These need to be taken out

Later we might come back and deal with this dataset if we can find a good use for it. But for now we will drop it. 

In [ ]:
# gallons_test = routine_full

In [ ]:
# gallons_test.dropna(subset=['Gallons'], inplace=True)

In [ ]:
# gallons_test.info()

There are a number of inch measurements in the gallons column. Since these are not useful, we will identify the inch variables and conver them to nan.

In [ ]:
# gallons_full = routine_full.copy()

In [ ]:
# gallons_full.loc[gallons_full['Gallons'].str.contains('"')==True] = np.nan

In [ ]:
# gallons_full.drop(['Sample', 'Prod', 'DateSampled', 'Grade', 'Supplier',
#        'FacilityName', 'SiteAddress', 'Test', 'Units', 'Method', 'Result',
#        'MinResult', 'MaxResult', 'Compliance'], axis=1, inplace=True)

In [ ]:
# routine_full['Gallons'] = gallons_full

In [ ]:
# routine_full['Gallons'] = routine_full['Gallons'].str.replace(',', '')

In [ ]:
routine_full.drop(['Gallons'], axis=1, inplace=True)

#### FacilityName

In [ ]:
routine_full['FacilityName'].isna().sum()

There are only 39 rows without a facility name. Since the number is low I will drop the rows without facility 

In [ ]:
routine_full.dropna(subset=['FacilityName'], inplace=True)

#### SiteAddress

In [ ]:
routine_full.loc[routine_full['FacilityName']=="FILL 'N' SAVE #3"]

Ask John why there are missing addresses

In [ ]:
routine_full[routine_full['SiteAddress'].isna()==True]

#### Complaince

Test contains 5 unique values, 'Y', ' ', 'N', nan, and 'Select'. From speaking with our expert we know that ' ', nan and select are all the same (none values. Not all tests are for compliance purposes). So we will make a seperate catagory called None. 

In [ ]:
routine_full['Test'].isna().sum()

In [ ]:
routine_full.Compliance.unique()

In [ ]:
routine_full['Compliance'] = routine_full['Compliance'].str.replace(' ', 'None')

In [ ]:
routine_full.Compliance.unique()

In [ ]:
routine_full['Compliance'] = routine_full['Compliance'].str.replace('Select', 'None')

In [ ]:
routine_full.Compliance.unique()

In [ ]:
routine_full['Compliance'] = routine_full['Compliance'].replace(np.nan, 'None')

In [ ]:
routine_full.Compliance.unique()

Let's reduce the tests we are working with to Distillation 50%, Vapor Pressure and Vapor-Liquid Ratio

In [ ]:
# routine_test_key = routine_full.loc[(routine_full['Test']=='Distillation 50%') 
#                  | (routine_full['Test']=='Vapor Pressure') 
#                  | (routine_full['Test']=='Vapor-Liquid Ratio')]

In [ ]:
# routine_test_key.to_csv('../../data/02_intermediate/routine_clean.csv', index=False)
routine_full.to_csv('../../data/02_intermediate/routine_clean.csv', index=False)